In [1]:
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import tabula
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO
import pandas as pd
import numpy as np

In [9]:
f = "bob7.pdf"
try:
    pars = tabula.read_pdf(f,                                            
                           pages='all',
                           silent=True,
                           lattice = True,
                           multiple_tables = True,
                           pandas_options={ 'header': None }  )          

    cust_info = pars[0]
    dic = cust_info.to_dict()
    
    #print(pars)

    for k in [ c for c in pars if c.shape[1]>=6 and c.shape[0]>2]:
        #k.drop(k.nunique(dropna=False)[(k.nunique(dropna=False) == 2)].index, axis=1,inplace=True)
        k.columns=list(range(k.shape[1]))    
        print(k.columns)
               
    df=pd.DataFrame()  
    try:
        df=pd.concat([ c for c in pars if c.shape[1]>=6 and c.shape[0]>2 and c.iloc[0,0] !='Type of Account']).drop_duplicates() 
    except: pass
    try:
        df=pd.concat([ c for c in pars if c.shape[1]==9 and c.shape[0]==1 and c.iloc[0,0] !='Type of Account']).drop_duplicates() 
    except: pass
    try:
        df=pd.concat([ c for c in pars if c.shape[1]==5 and c.shape[0]>2 and c.iloc[0,0] !='Type of Account']).drop_duplicates() 
    except: pass
    print("Parsed")
    #print(df)
except:
    print("Not Parsed")
    pass


try:
    df=df.replace(r'^\s*$', np.nan, regex=True)
    df=df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)       
    df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)
    try:
        idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
        df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True) 
        print("Columns:",df.columns)
    except:
        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
            df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)           
        except:
            try:
                idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('narration').any(), axis=1) ==True].index ][0]
                df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(    drop=True,inplace=True)           
            except:
                print("\nProcess-Column headers missing") ; log.write("\nProcess-Column headers missing") 
    
    df=df[~df.index.isin(df[df.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
    df=pd.DataFrame(df.T.drop_duplicates().T)
    df.drop(df.nunique(dropna=False)[(df.nunique(dropna=False) == 1)].index, axis=1,inplace=True)
         
    try:
        dft=pd.DataFrame(df.iloc[:,0].dropna().unique().astype(int)).reset_index()
        if sum(dft.iloc[:,1])-sum(dft.iloc[:,0])==dft.shape[0] :
            df.drop(df.iloc[:,0].name, axis=1,inplace=True)
    except:  pass
    df=df.drop_duplicates()
    print("Total Parsed")
except:
    print("Can't parsed")
    pass

try:
    df.to_csv("parsed_bob.csv", index=0)
except:
    print("CSV not Create.")
    pass

Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')
Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')
Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')
Parsed
Columns: Index(['S.No', 'Date', 'Description', 'Cheque No.', 'Debit', 'Credit',
       'Balance', 'Value Date'],
      dtype='object', name=0)
Total Parsed
